# Import required libraries

In [1]:
# import basic libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# import libraries for machine learning models
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

# import libraries to solve LP
from pulp import *

# Output $p_i$ and $q_i$ for each candidate i $\in$ [n]

### Train the predict algorithm

In [2]:
# read data
df = pd.read_csv('clean_law_school.csv', index_col = 0)

# split data into training and testing part
target = ['admit', 'enroll']
y = df[target]
X = df.drop(target, axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, shuffle = True, random_state = 1)

# implement the machine learning model to predict p_i and q_i
lg_clf = ClassifierChain(RandomForestClassifier())
lg_clf.fit(X_train, y_train)

ClassifierChain(classifier=RandomForestClassifier(), require_dense=[True, True])

### Output $p_i$ and $q_i$ for each i in the test data

In [3]:
y_pred = lg_clf.predict_proba(X_test)
X_test = pd.merge(X_test, y_test, left_index = True, right_index = True)
X_test[['p_i', 'q_i']] = np.round(y_pred.toarray(), 3)
X_test

,lsat,gpa,resident,gender,black,hispanic,asian,white,other_race,c_admit_rate,c_enroll_rate,es,admit,enroll,p_i,q_i
54066,150.0,2.77,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.649,0.381,0.323,0.0,0.0,0.110,0.00
59441,155.0,2.81,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000,0.285,0.151,0.0,0.0,0.010,0.00
11354,161.0,3.78,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.219,0.180,0.108,1.0,0.0,0.500,0.59
68562,164.0,3.44,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.361,0.149,0.387,1.0,0.0,0.590,0.04
97741,149.0,3.79,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.642,0.405,0.280,0.0,0.0,0.090,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51882,153.0,3.05,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.649,0.381,0.323,0.0,0.0,0.070,0.00
62536,162.0,3.57,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000,0.285,0.151,1.0,1.0,0.608,0.37
98995,156.0,3.30,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.638,0.740,0.500,1.0,1.0,0.960,0.97
6304,152.0,3.22,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.693,1.000,0.301,0.0,0.0,0.010,0.00


### Note on the selection of candidates for a given department

In APD-S, since we focus on one academic unit (e.g, department), we would only select applicants from colleges with similar admit rate and enroll rate (this means that we can form 1 department out of those schools) for the input instance.

In [4]:
# summary of college admit rate
X_test['c_admit_rate'].describe()

count    22754.000000
mean         0.347415
std          0.238890
min          0.000000
25%          0.192000
50%          0.315000
75%          0.432000
max          1.000000
Name: c_admit_rate, dtype: float64

In [5]:
# summary of college enroll rate
X_test['c_enroll_rate'].describe()

count    22754.000000
mean         0.356386
std          0.237919
min          0.000000
25%          0.180000
50%          0.285000
75%          0.433000
max          1.000000
Name: c_enroll_rate, dtype: float64

Based on the summary of the statistics of colleges' admit and enroll rate, we would select applicants from college in the range of 25th percentile and 75th percentile.

In [6]:
X_test = X_test[(X_test['c_admit_rate'] >= np.percentile(X_test['c_admit_rate'], 25)) & \
                (X_test['c_admit_rate'] <= np.percentile(X_test['c_admit_rate'], 75)) & \
                (X_test['c_enroll_rate'] >= np.percentile(X_test['c_enroll_rate'], 25)) & \
                (X_test['c_enroll_rate'] <= np.percentile(X_test['c_enroll_rate'], 75))]

# Generate input instance for APD-S

### Specification
An input instance of APD can be characterized as $I = ([n], \{p_i, q_i |i \in [n]\}, \{B_g, g|g \in G_I\}, \{b_g, g|g \in G_E\}, \{w_{ig}, \tau_g|g \in G_P, i \in g\})$.

**Note:** 
- In this dataset, we assume all candidates are qualified for an interview and they will be automatically offered once passing it. Hence, it can be explained why we use the result of *admit* as *passing the interview* and *enroll* as *accepting the offer*.

**Input details:**
- [n] = {1000, 2000, 3000, 4000, 5000}
- $p_i, q_i$ for each i $\in$ [n]
- $G_I$:
    - In APD-S, we only have single interview-related constraint that can be captured as {g, B} with g = [n] being the only group in $G_I$. So $G_I$ contains only 1 group with n candidates.
    - In reality, the dataset should miss a lot of candidates that fail to get an interview since it only includes those who qualify for an interview. In this case, we can simulate how colleges with similar acceptance rate (~20-30%) as schools in this data perform. We found [Admissions report of Oxford Law](https://www.law.ox.ac.uk/sites/files/oxlaw/ug_admissions_report_2021.pdf) and identified that we can use its application-to-interview success rate to set the cap on interview-related group g $B_g$ accordingly.
- $G_E$:
    - We have enrollment-related budget constraints {g, $b_g|g \in G_E$}. In this dataset, there are two groups inside $G_E$ which indicate candidates who are in-state and those who are out-of-state.
    - Since the data should not miss any candidate who successfully enrolls, we can set the cap on enrollment-related group g $b_g$ as the actual statistics of the dataset (of those who enroll, who are in-state applicants, who are out-of-state applicants?)
- $G_P$:
    - We capture the collection of protected groups of interest $G_P$ as the combination of the race and gender of the candidates. That means $G_P$ will have 8 groups by combining Race: {Black, Hispanic, Asian, White} and Gender: {Male, Female}.
    - We identify the target quota $\tau_g$ for protected group g using the admission statistics of universities that are known for applying Affirmative Action in their admission process. 
        + Specifically, we would use the statistics of Harvard Law School, known for its [yearly commitment to Affirmative Action in the admission/employment process](https://hr.harvard.edu/files/humanresources/files/reaffirmation_statement.pdf)
        + Based on the [demographics of Hardvard Fall 2020 applications](https://www.ilrg.com/rankings/law/view/49), we calculate the percentage of each protected group in the enrollment number and set $\tau_g$ accordingly. 
    - For each candidate i $\in g$ of $G_P, w_{ig}$ (the degree of relevance of i to g) is calculated by the economic status of the candidate. We use the school's location to reflect the poverty rate - an important indicator for economic status.

### Define n candidates and randomly sample a data of size n from the test data

In [7]:
# create 5 random input instances from the testing data set, each with {1000, 2000, 3000, 4000, 5000} candidates
sizes = np.array([n * 1000 for n in range(1, 6)])
input_instances = np.array([X_test.sample(n = n).reset_index(drop = True) for n in sizes], dtype = object)

### Function to generate input instance

In [8]:
def generate_input(data):
    # collection of interview-related groups
    G_I = data
    
    # collection of enrollment-related groups
    in_state = (data['resident'] == 1)
    out_of_state = (data['resident'] == 0)
    G_E = np.array([data[in_state], data[out_of_state]], dtype = object)
    
    # collection of protected groups
    female_black = data[(data['gender'] == 0) & (data['black']==1)]
    female_hispanic = data[(data['gender'] == 0) & (data['hispanic'] == 1)]
    female_asian = data[(data['gender'] == 0) & (data['asian'] == 1)]
    female_white = data[(data['gender'] == 0) & (data['white'] == 1)]
    female_other = data[(data['gender'] == 0) & (data['other_race'] == 1)]
    
    male_black = data[(data['gender'] == 1) & (data['black'] == 1)]
    male_hispanic = data[(data['gender'] == 1) & (data['hispanic'] == 1)]
    male_asian = data[(data['gender'] == 1) & (data['asian'] == 1)]
    male_white = data[(data['gender'] == 1) & (data['white'] == 1)]
    male_other = data[(data['gender'] == 1) & (data['other_race'] == 1)]

    G_P = np.array([female_black, female_hispanic, female_asian, female_white, female_other,\
           male_black, male_hispanic, male_asian, male_white, male_other], dtype = object) 
    
    # cap imposed on interview-related group g
    B_g = int(len(data) * 0.3705)
    
    # cap imposed on enrollment-related group g
    b_g = np.array([len(data[in_state & (data['enroll'] == 1)]), len(data[out_of_state & (data['enroll'] == 1)])])
    
    # target quota for protected group g to achieve
    target_quota = [0.0345, 0.0415, 0.0535, 0.252, 0.1185] * 2
    tau_g = np.array(np.round([len(data[data['enroll'] == 1]) * quota for quota in target_quota]), int)
    
    # if there is a difference in the sum of G_P and G_E due to rounding, randomly increase one group
    # in either G_P or G_E to balance the difference (as both cap on final enrollment)
    if sum(tau_g) > sum(b_g):
        index = np.random.randint(0, len(b_g)) 
        b_g[index] += sum(tau_g) - sum(b_g)
    elif sum(tau_g) < sum(b_g):
            index = np.random.randint(0, len(tau_g))
            tau_g[index] += sum(b_g) - sum(tau_g)
    
    # relevance of i to protected group g
    w_ig = []
    for index_g in range(len(G_P)):
        g = G_P[index_g]
        arr = []
        for index_i in range(len(g)):
            arr.append(g.iloc[index_i]['es'])
        w_ig.append(arr)
    w_ig = np.asarray(w_ig, dtype = object)
    
    return G_I, G_E, G_P, B_g, b_g, tau_g, w_ig

# Solve Linear Programming

### Justification for solving the maximin LP

The objective model is max min$_{g \in G_P} (\sum_{i \in g} w_{ig} y_i q_i / \tau_g)$.

We can rewrite it as the following to solve:

max z

s.t $ \space \space$  z $\le \sum_{i \in g} w_{ig} y_i q_i / \tau_g \space \space \space \space \space \space$ for $g \in G_P$

Other constraints will be kept as original.

### Function to solve LP

In [9]:
def solveLP(data, n, G_I, G_E, G_P, B_g, b_g, tau_g, w_ig):
    # create model
    model = LpProblem(name='APD-S', sense = LpMaximize)

    # define decision variables
    x = np.array([LpVariable('x' + str(i), lowBound = 0, upBound = 1) for i in range(n)])
    y = np.array([LpVariable('y' + str(i), lowBound = 0, upBound = 1) for i in range(n)])
    z = LpVariable(name='z')

    # add objective function to the model
    model += z

    # constraints for z
    for index_g in range(len(G_P)):
        constraint = []
        g = G_P[index_g]
        for index_i in range(len(g)):
            constraint.append(w_ig[index_g][index_i]*y[g.index[index_i]]*g.iloc[index_i]['q_i']/tau_g[index_g])
        model += z <= lpSum(constraint)
    
    # constraints for (2) & (3) in LP since G_I = [n]
    constraint = []
    for i in range(n):
        # (3)
        model += y[i] <= x[i]*data.iloc[i]['p_i']
        # (2)
        constraint.append(x[i])  
    model += lpSum(constraint) <= B_g
        
    # constraints for (4) in LP
    for index_g in range(len(G_E)):
        g = G_E[index_g]
        constraint = []
        for index_i in range(len(g)):
            constraint.append(y[g.index[index_i]]*g.iloc[index_i]['q_i'])
        model += lpSum(constraint) <= b_g[index_g]
    
    # solve the model 
    model.solve(PULP_CBC_CMD(msg=0))
    
    x_optimal = np.zeros(n)
    y_optimal = np.zeros(n)
    
    for var in model.variables()[:n]:
        x_optimal[int(str(var.name)[1:])] = round(var.varValue, 3)
    
    for var in model.variables()[n:-1]:
        y_optimal[int(str(var.name)[1:])] = round(var.varValue, 3)
    
    # return x*_i, y*_i for each candidate i
    return x_optimal, y_optimal

# Implementation of algorithm 1 for the special case of APD-S

In [10]:
def algorithm_1(data, n, G_E, B_g, b_g, x_optimal, y_optimal):
    # remarks on RP for APD-S
    L_i = np.empty(n, dtype = object)
    T_l = np.empty([n, B_g])
    T_l_prime = np.empty([n, B_g])
    for i in range(n):
        # generate interval L_i for each i
        left = sum(x_optimal[:i])
        right = sum(x_optimal[:(i+1)])
        L_i[i] = pd.Interval(left = left, right = right, closed = 'both')

        # generate B i.i.d T_l and T'_l for each l
        l = i
        T_l[l] = np.random.uniform(0, np.nextafter(1, np.inf), B_g)
        T_l_prime[l] = np.array([l - 1 + T for T in T_l[l]])

    # set I_i = 1 if there exists at least one l in n with T'_l in L_i
    I_i = np.zeros(n)
    for i in range(n):
        for l in range(n):
            if all(num in L_i[i] for num in T_l_prime[l]): # if T'_l in L_i
                I_i[i] = 1
                break

    # step (5) - (11):
    pi = np.random.permutation(n)
    Z_i = np.zeros(n)
    sum_Zi = [0, 0]
    for l in range(n):
        i = pi[l]
        if I_i[i] == 1: # give interview to i
            p_i = data.iloc[i]['p_i']
            if np.random.choice([True, False], size = 1, p = [p_i, 1 - p_i]): # if i passes the interview
                O_i =  np.random.binomial(1, y_optimal[i]/(x_optimal[i] * p_i))
                if O_i == 1: 
                    for index_g in range(len(G_E)):
                        # locate i in which group g of G_E
                        if i in G_E[index_g].index: 
                            if sum_Zi[index_g] <= b_g[index_g]:
                                sum_Zi[index_g] += 1
                                Z_i[i] = 1         
                            break
    
    return Z_i

# Verify LP constraints

### Function to verify the constraints of LP

In [11]:
# verify if x* and y* of each input instance satisfy the constraints 
def verify_LP(data, n, G_E, B_g, b_g, x_optimal, y_optimal):
    print('Verify input instance with n =', n)

    # constraint 2
    if sum(x_optimal) > B_g:
        print('Not satisfied!')
        return False

    # constraint 3
    for i in range(n):
        if y_optimal[i] > x_optimal[i]*data.iloc[i]['p_i']:
            print('Not satisfied!')
            return False

    # constraint 4
    for index_g in range(len(G_E)):
        g = G_E[index_g]
        for index_i in range(len(g)):
            if y_optimal[g.index[index_i]]*g.iloc[index_i]['q_i'] > b_g[index_g]:
                print('Not satisfied!')
                return False
            
    # if no constraint is violated 
    print('-> Every constraints has been satisfied!')

### Verify LP with all input instances

In [12]:
for i in range(len(input_instances)):
    # generate input instance
    data = input_instances[i]
    n = sizes[i]
    G_I, G_E, G_P, B_g, b_g, tau_g, w_ig = generate_input(data)
              
    # output x*_i, y*_i for each i in [n]
    x_optimal, y_optimal = solveLP(data, n, G_I, G_E, G_P, B_g, b_g, tau_g, w_ig)
    
    verify_LP(data, n, G_E, B_g, b_g, x_optimal, y_optimal)

Verify input instance with n = 1000
-> Every constraints has been satisfied!
Verify input instance with n = 2000
-> Every constraints has been satisfied!
Verify input instance with n = 3000
-> Every constraints has been satisfied!
Verify input instance with n = 4000
-> Every constraints has been satisfied!
Verify input instance with n = 5000
-> Every constraints has been satisfied!


### Verify Lemma 2: $E[Z_i] \ge (y_i^{*}.q_i)/2$ for every $i \in n$

### Function to verify Lemma 2

In [13]:
def verify_lemma2(data, n, G_E, B_g, b_g, x_optimal, y_optimal):
    print('Verify Lemma 2 with input instance that has n =', n)
    expected_Z = np.zeros(n)
    for i in range(200):
        Z_i = algorithm_1(data, n, G_E, B_g, b_g, x_optimal, y_optimal)
        expected_Z += Z_i
    expected_Z /= 200
    flag = True
    for i in range(n):
        if expected_Z[i] < (y_optimal[i]*data.iloc[i]['q_i'])/2:
            print('Error:')
            print('E[Z_i] =', expected_Z[i])
            print('(y*_i * q_i)/2 =', (y_optimal[i]*data.iloc[i]['q_i'])/2)
            flag = False
    if flag:
        print('Lemma 2 has been verified!')
    else:
        print('Lemma 2 has not been verified')

### Verify Lemma 2

In [ ]:
for i in range(len(input_instances)):
    # generate input instance
    data = input_instances[i]
    n = sizes[i]
    G_I, G_E, G_P, B_g, b_g, tau_g, w_ig = generate_input(data)
              
    # output x*_i, y*_i for each i in [n]
    x_optimal, y_optimal = solveLP(data, n, G_I, G_E, G_P, B_g, b_g, tau_g, w_ig)
              
    verify_lemma2(data, n, G_E, B_g, b_g, x_optimal, y_optimal)

Verify Lemma 2 with input instance that has n = 1000


# Output Approximation Ratio for RP

# Greedy algorithm

# Uniform algorithm

# Visualize the confidence interval of the Approximation Ratio for each algorithm